 ___Sentiment Analysis on tomato Reviews using Random and TF-IDF Approach.___

__Feature Extraction__: TF-IDF (term frequency-inverse document frequency)

__Classification__: SVM, Logistic Regression 

In this notebook, Sentiment Analysis is performed on dataset.

Import the Required Packages:

In [41]:
!pip install wandb -qU

In [42]:
import wandb
wandb.login(key="aec6fef7ba56ee445129472eb583718b8e529934")

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

In [43]:
import gzip
import pickle
import pandas as pd
import numpy as np
import nltk
import re
import random
from tqdm import tqdm
import json
import wandb
from sklearn.metrics import classification_report

In [44]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [45]:
%cd /content/drive/MyDrive/mlops/mlops_part_2_progect/reviews

/content/drive/MyDrive/mlops/mlops_part_2_progect/reviews


In [46]:

with open('train.pkl', 'rb') as f:
    train_set = pickle.load(f)
    
with open('test.pkl', 'rb') as f:
    test_set = pickle.load(f)
        
 

In [47]:
train = pd.DataFrame.from_dict(train_set)
test =  pd.DataFrame.from_dict(test_set)

In [48]:
train = train.dropna()
test = test.dropna()

In [49]:
X_train = train["x"]
X_test = test["x"]
Y_train = train["y"]
Y_test = test["y"]


# Random
 <u><i>motivation</i></u>: We want to know what random (chance) performance looks like. All of our efforts should be well above this baseline.

In [50]:
# Generate random predictions

label = ["Fresh","Rotten"]
y_pred = []
for i in range(len(Y_test)):
    y_pred.append(random.choice(label))

from sklearn.metrics import classification_report
metrics=classification_report(Y_test, y_pred,output_dict=True)


In [51]:
from sklearn.metrics import classification_report
metrics=classification_report(Y_test, y_pred,output_dict=True)
performance = {"accuracy": metrics["accuracy"],
               "precision": metrics["weighted avg"]['precision'],
               "recall": metrics["weighted avg"]['recall'],
               "f1": metrics["weighted avg"]['f1-score']}

In [52]:
performance = {"accuracy": metrics["accuracy"],
               "precision": metrics["weighted avg"]['precision'],
               "recall": metrics["weighted avg"]['recall'],
               "f1": metrics["weighted avg"]['f1-score']}

In [53]:
print (json.dumps(performance, indent=2))

{
  "accuracy": 0.4995996291301416,
  "precision": 0.5388102853049039,
  "recall": 0.4995996291301416,
  "f1": 0.5095863722381182
}


By Using the __random__ classifier we got an accuracy of 50.03%

__weighted random__

We made the assumption that there is an equal probability for every class. Let's use the train split to figure out what the true probability is.

In [54]:
# Class frequencies
import collections
collections.Counter(Y_test)

Counter({'Fresh': 60733, 'Rotten': 34179})

In [55]:
p=[]
for i in label:
     p.append(collections.Counter(Y_test)[i]/len(Y_test))

In [56]:
y_pred = []
for i in range(len(Y_test)):
    y_pred.append(random.choices(label, weights=p ,k=1) )

In [57]:
metrics_weighted_random = classification_report(Y_test, y_pred,output_dict=True)
performance_weighted_random = {"accuracy": metrics["accuracy"],
               "precision": metrics["weighted avg"]['precision'],
               "recall": metrics["weighted avg"]['recall'],
               "f1": metrics["weighted avg"]['f1-score']}
print (json.dumps(performance, indent=2))


{
  "accuracy": 0.4995996291301416,
  "precision": 0.5388102853049039,
  "recall": 0.4995996291301416,
  "f1": 0.5095863722381182
}


By Using the __weighted random__ classifier we got an accuracy of 53.93%

# Feature Extraction
__TF-IDF__: It is a method of extracting the features from the text data. TF stands for Term Frequency and IDF stands for Inverse Document Frequency.

__Term Frequency__: Number of times word occurs in a review. For an example consider 2 reviews where w1,w2.. represents the words in both reviews and table defines the frequency of words in the particular review.

IDF is computed as :

    idf(t) = log [ n / df(t) ] + 1
         = log [ number of documents / number of documents containing the term]+1
         
         
If smooth_idf=True.
        Smooth-IDF = log [ n / df(t) +1 ] + 1         


TF-IDF is implemented using sklearn : [link](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html)




__Using TF*IDF Vectorizer__

In [58]:
print("TFIDF Vectorizer……")
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer= TfidfVectorizer()
tf_x_train = vectorizer.fit_transform(X_train)
tf_x_test = vectorizer.transform(X_test)

TFIDF Vectorizer……


__SVM__

Implementing SVM with sklearn for classification

In [59]:
from sklearn.svm import LinearSVC
clf = LinearSVC(random_state=0,verbose=True)

In [60]:
clf.fit(tf_x_train,Y_train)

[LibLinear]

LinearSVC(random_state=0, verbose=True)

Predicting the Test data

In [61]:
y_test_pred=clf.predict(tf_x_test)

In [62]:
metrics_SVM=classification_report(Y_test, y_test_pred,output_dict=True)

In [63]:
performance_SVM = {"accuracy": metrics["accuracy"],
               "precision": metrics["weighted avg"]['precision'],
               "recall": metrics["weighted avg"]['recall'],
               "f1": metrics["weighted avg"]['f1-score']}
print (json.dumps(performance, indent=2))

{
  "accuracy": 0.4995996291301416,
  "precision": 0.5388102853049039,
  "recall": 0.4995996291301416,
  "f1": 0.5095863722381182
}


__By Using the SVM classifier we got an accuracy of 80.87%__









__Logistic Regression__

Logistic regression is implemented using sklearn


In [64]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(max_iter=1000,solver="saga")

Fit the Training data to the model

In [65]:
clf.fit(tf_x_train,Y_train)

LogisticRegression(max_iter=1000, solver='saga')

Predicting the test data

In [66]:
y_test_pred_Logistic_Regression=clf.predict(tf_x_test)

Analyzing the Report

In [67]:
metrics_Logistic_Regression = classification_report(Y_test, y_test_pred,output_dict=True)

In [68]:
performance_Logistic_Regression = {"accuracy": metrics["accuracy"],
               "precision": metrics["weighted avg"]['precision'],
               "recall": metrics["weighted avg"]['recall'],
               "f1": metrics["weighted avg"]['f1-score']}
print (json.dumps(performance, indent=2))

{
  "accuracy": 0.4995996291301416,
  "precision": 0.5388102853049039,
  "recall": 0.4995996291301416,
  "f1": 0.5095863722381182
}


__By Using the LR classifier we got an accuracy of 81.27%__

In [69]:
# Initialize wandb project
wandb.init(project="svm-project")



# Log the metrics and models to wandb
wandb.log({

            "accuracy_weighted_random": metrics_weighted_random["accuracy"],
           "precision_weighted_random": metrics_weighted_random["weighted avg"]['precision'],
           "recall_weighted_random": metrics_weighted_random["weighted avg"]['recall'],
           "f1_weighted_random": metrics_weighted_random["weighted avg"]['f1-score']
            ,

           "accuracy_svm": metrics_SVM["accuracy"],
           "precision_svm": metrics_SVM["weighted avg"]['precision'],
           "recall_svm": metrics_SVM["weighted avg"]['recall'],
           "f1_svm": metrics_SVM["weighted avg"]['f1-score']
           ,
   
           "accuracy_Logistic_Regression": metrics_Logistic_Regression["accuracy"],
           "precision_Logistic_Regression": metrics_Logistic_Regression["weighted avg"]['precision'],
           "recall_Logistic_Regression": metrics_Logistic_Regression["weighted avg"]['recall'],
           "f1_Logistic_Regression": metrics_Logistic_Regression["weighted avg"]['f1-score']
           })

wandb.sklearn.plot_confusion_matrix(Y_test, y_test_pred, label)

wandb.sklearn.plot_confusion_matrix(Y_test, y_test_pred_Logistic_Regression, label)

accuracy_Logistic_Regression,▁
accuracy_svm,▁
accuracy_weighted_random,▁
f1_Logistic_Regression,▁
f1_svm,▁
f1_weighted_random,▁
precision_Logistic_Regression,▁
precision_svm,▁
precision_weighted_random,▁
recall_Logistic_Regression,▁
recall_svm,▁
